# Proximal Policy Optimization (PPO): The Modern Standard

## 🎯 From Actor-Critic TD to PPO: The Evolution of Policy Gradient Methods

Welcome to **Proximal Policy Optimization (PPO)** - the algorithm that has dominated reinforcement learning since 2017 and remains the go-to method even in 2025. PPO represents the culmination of decades of research into stable, sample-efficient policy gradient methods.

## 📈 The Historical Journey: REINFORCE → Actor-Critic → TRPO → PPO

### The Problem with Vanilla Policy Gradients

From our previous notebooks, we've seen the progression:

1. **REINFORCE**: High variance, simple implementation
2. **Actor-Critic MC**: Reduced variance with baselines, but still episode-based
3. **Actor-Critic TD**: Bootstrapping for sample efficiency, but training instability
4. **A2C**: Added parallel environments for stability and speed

**The Core Challenge**: All these methods suffer from **destructive policy updates** - a single bad gradient step can destroy hours of learning progress.

### Trust Region Policy Optimization (TRPO): The Breakthrough

**TRPO (2015)** solved the destructive update problem with a brilliant insight:

**Core Idea**: Constrain policy updates to stay within a "trust region" where our gradient estimates are reliable.

**Mathematical Formulation**:
$$\min_\theta -\mathbb{E}[L_{\text{maximize}}^{TRPO}(\theta)] \text{, subject to } \mathbb{E}[KL(\pi_{\theta_{old}}, \pi_\theta)] \leq \delta$$

Where:
- $L_{\text{maximize}}^{TRPO}(\theta)$ is the surrogate objective (importance sampling)
- $KL(\pi_{\theta_{old}}, \pi_\theta)$ is the KL divergence between old and new policies
- $\delta$ is the trust region constraint

**TRPO's Innovation**: 
- **Monotonic improvement**: Guaranteed to never make the policy worse
- **Stable learning**: Prevents destructive updates through KL constraint
- **Theoretical guarantees**: Provable convergence properties

**TRPO's Fatal Flaw**: 
- **Computational complexity**: Requires second-order optimization (natural gradients)
- **Difficult implementation**: Complex conjugate gradient and line search procedures
- **Slow**: Expensive computation per update step

### PPO: The Practical Solution (Similar to A3C → A2C Evolution)

**PPO (2017)** achieved TRPO's benefits with a simple, efficient implementation, following a similar pattern to how **A2C simplified A3C**:

**The Simplification Pattern in RL**:
- **A3C → A2C**: Asynchronous complexity → Synchronous simplicity
- **TRPO → PPO**: Constrained optimization complexity → Clipped objective simplicity

**Key Insight**: Instead of constraining KL divergence, **clip the objective function** to prevent large updates.

**Why PPO Won (Echoing A2C's Success)**:
- **Simple implementation**: First-order optimization only (like A2C's synchronous updates)
- **Computational efficiency**: Fast and scalable (like A2C's GPU-friendly batching)
- **Robust performance**: Works well across diverse environments
- **Stable learning**: Prevents destructive updates like TRPO
- **Sample efficiency**: Reuses data through multiple epochs
- **Engineering principle**: **Simpler objectives often win in computer science**

**PPO's Dominance (2017-2025)**:
- **OpenAI's choice**: Used for ChatGPT, GPT-4, and other large-scale RL applications
- **Industry standard**: Default choice for most RL practitioners
- **Research baseline**: Standard comparison algorithm in RL papers
- **Continued relevance**: Still the best general-purpose RL algorithm in 2025

## 🔧 PPO's Key Innovations

PPO builds upon Actor-Critic TD with parallel environments (like A2C) and adds crucial stability improvements:

### 1. 🎯 Clipped Surrogate Objective

**Problem**: Standard policy gradients can make arbitrarily large updates, destroying learning progress.

**Why Do We Need the Probability Ratio? Understanding Importance Sampling**

**The Data Reuse Problem**: In standard Actor-Critic TD, we collect data with policy $\pi_{\theta_{old}}$ but want to update to $\pi_\theta$. When we reuse this "old" data for multiple training epochs, we're evaluating a **different policy** than the one that generated the data.

**Standard Policy Gradient (On-Policy)**:
$$\nabla_\theta J(\theta) = \mathbb{E}_{s \sim \rho_{\pi_\theta}, a \sim \pi_\theta}[\nabla_\theta \log \pi_\theta(a|s) A^{\pi_\theta}(s,a)]$$

**The Challenge**: This expectation assumes data comes from the **current** policy $\pi_\theta$, but our data comes from the **old** policy $\pi_{\theta_{old}}$.

**Importance Sampling to the Rescue**: We can correct for this mismatch using importance sampling - a technique that lets us estimate expectations under one distribution using samples from another.

**Importance Sampling Formula**:
$$\mathbb{E}_{x \sim p}[f(x)] = \mathbb{E}_{x \sim q}\left[\frac{p(x)}{q(x)} f(x)\right]$$

Where:
- $p(x)$ is the distribution we want to estimate from
- $q(x)$ is the distribution we actually have samples from
- $\frac{p(x)}{q(x)}$ is the **importance weight** that corrects the bias

**Applied to Policy Gradients**:
$$\nabla_\theta J(\theta) = \mathbb{E}_{s \sim \rho, a \sim \pi_{\theta_{old}}}\left[\frac{\pi_\theta(a|s)}{\pi_{\theta_{old}}(a|s)} \nabla_\theta \log \pi_\theta(a|s) A(s,a)\right]$$

**The Probability Ratio**:
$$r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$$

**Intuitive Meaning of the Ratio**:
- **$r_t = 1.0$**: New policy assigns same probability as old policy (no change)
- **$r_t > 1.0$**: New policy more likely to take this action than old policy
- **$r_t < 1.0$**: New policy less likely to take this action than old policy

**Why This Works - The Learning Signal Mechanism**: 

The objective we optimize is: $L = \mathbb{E}[r_t(\theta) \cdot A_t]$

**Case 1: Good Action (Positive Advantage)**
- $A_t > 0$ means this action was better than expected
- We want to **increase** the probability of taking this action
- If $r_t > 1$: New policy already favors this action → $r_t \cdot A_t > A_t$ → **strong positive signal**
- If $r_t < 1$: New policy disfavors this action → $r_t \cdot A_t < A_t$ → **weak positive signal**
- **Result**: Gradient pushes to increase probability of good actions

**Case 2: Bad Action (Negative Advantage)**
- $A_t < 0$ means this action was worse than expected  
- We want to **decrease** the probability of taking this action
- If $r_t > 1$: New policy favors bad action → $r_t \cdot A_t <$ (more negative) → **strong negative signal**
- If $r_t < 1$: New policy already disfavors bad action → $r_t \cdot A_t >$ (less negative) → **weak negative signal**
- **Result**: Gradient pushes to decrease probability of bad actions

**Concrete Example**:
- **Action**: Move left in state S
- **Old policy**: $\pi_{\text{old}}(\text{left}|S) = 0.2$ (20% probability)
- **Advantage**: $A = +5$ (good action!)
- **New policy option 1**: $\pi_{\text{new}}(\text{left}|S) = 0.4$ → $r_t = 0.4/0.2 = 2.0$
- **New policy option 2**: $\pi_{\text{new}}(\text{left}|S) = 0.1$ → $r_t = 0.1/0.2 = 0.5$

**Learning signals**:
- **Option 1**: $2.0 \times (+5) = +10$ (strong positive gradient)
- **Option 2**: $0.5 \times (+5) = +2.5$ (weak positive gradient)

**The magic**: Option 1 gets rewarded more because it's **already moving in the right direction** (increasing probability of the good action), while Option 2 gets a weaker signal because it's moving the wrong way.

**Standard Policy Gradient with Importance Sampling (as a loss to minimize)**:
$$L^{PG}(\theta) = -\mathbb{E}\left[\sum_{t=0}^{T-1} r_t(\theta) A_t\right]$$

**The Problem with Unbounded Ratios**: If $r_t(\theta)$ becomes very large (policy changes dramatically), the gradient can become unstable and destructive.

**PPO's Solution: Clip the Ratio**

Instead of letting the ratio go to infinity, PPO **clips** it to a safe range:

$$L^{CLIP}(\theta) = -\mathbb{E}\left[\sum_{t=0}^{T-1} \min(r_t(\theta) A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) A_t)\right]$$

**Step-by-Step Breakdown**:

1. **Compute probability ratio**: $r_t = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$

2. **Create clipped version**: $r_t^{clipped} = \text{clip}(r_t, 1-\epsilon, 1+\epsilon)$
   - If $r_t < 1-\epsilon$: $r_t^{clipped} = 1-\epsilon$
   - If $r_t > 1+\epsilon$: $r_t^{clipped} = 1+\epsilon$  
   - Otherwise: $r_t^{clipped} = r_t$

3. **Compute two objectives**:
   - **Original**: $r_t \cdot A_t$ (importance-sampled objective)
   - **Clipped**: $r_t^{clipped} \cdot A_t$ (conservative objective)

4. **Take the minimum**: $\min(r_t A_t, r_t^{clipped} A_t)$

**Why Take the Minimum? The Conservative Principle**

The $\min()$ operation implements a **pessimistic** strategy:

**Case 1: Positive Advantage ($A_t > 0$)**
- **Good action**: We want to increase its probability
- **If $r_t > 1+\epsilon$**: Clipping prevents excessive increase
- **Intuition**: "Don't get too excited about good actions"

**Case 2: Negative Advantage ($A_t < 0$)**  
- **Bad action**: We want to decrease its probability
- **If $r_t < 1-\epsilon$**: Clipping prevents excessive decrease
- **Intuition**: "Don't get too harsh on bad actions"

**Concrete Example**:
- **Old policy**: $\pi_{\theta_{old}}(a|s) = 0.1$ (10% chance)
- **New policy**: $\pi_\theta(a|s) = 0.5$ (50% chance)
- **Ratio**: $r_t = 0.5/0.1 = 5.0$
- **Advantage**: $A_t = +10$ (good action)
- **Clip epsilon**: $\epsilon = 0.2$ → clipped ratio = $1.2$

**Without clipping**: Objective = $5.0 \times 10 = 50$ (huge update!)  
**With clipping**: Objective = $\min(50, 1.2 \times 10) = \min(50, 12) = 12$ (moderate update)

**The Key Insight**: PPO only allows **moderate** policy changes, preventing the destructive updates that plague standard policy gradients while still enabling learning progress.

**Benefits of Clipped Objective**:
- **Stability**: Prevents destructive policy updates
- **Data efficiency**: Enables safe reuse of experience data
- **Simplicity**: No complex constrained optimization like TRPO
- **Robustness**: Works across diverse environments and hyperparameters

### 2. 🌐 Parallel Environment Collection

**Problem**: Single environment collection is slow and provides limited data diversity.

**Solution**: Use multiple parallel environments like A2C, but with PPO's rollout-based collection.

**Benefits**:
- **Faster data collection**: Multiple environments running simultaneously
- **Better batch diversity**: Each environment may be in different states
- **More stable gradients**: Averaging across diverse experiences
- **GPU efficiency**: Natural batching for neural network updates

### 3. 🎲 Generalized Advantage Estimation (GAE)

**Problem**: Actor-Critic TD still has high variance in advantage estimates.

**Solution**: Blend multiple n-step returns with exponential weighting.

**Standard N-Step Advantage**:
$$A_t^{(n)} = \sum_{k=0}^{n-1} \gamma^k r_{t+k+1} + \gamma^n V(s_{t+n}) - V(s_t)$$

**GAE Formula**:
$$A_t^{GAE(\lambda)} = \sum_{k=0}^{\infty} (\gamma \lambda)^k \delta_{t+k}$$

Where $\delta_t = r_{t+1} + \gamma V(s_{t+1}) - V(s_t)$ is the TD error.

**Effective Horizon Example**: With $\gamma = 0.99$ and $\lambda = 0.95$, the weights decay as $(\gamma \lambda)^k = 0.9405^k$. After ~15 steps, the weight drops to $0.9405^{15} \approx 0.1$, meaning GAE effectively considers about 15 future steps while theoretically extending to infinity.

**Benefits**:
- **Bias-variance tradeoff**: $\lambda=0$ (low variance, high bias) to $\lambda=1$ (high variance, low bias)
- **Flexible**: Can interpolate between TD and Monte Carlo methods
- **Efficient**: Exponential weighting reduces computational cost

### 4. 🌟 Entropy Regularization

**Problem**: Policies can converge prematurely to suboptimal solutions.

**Solution**: Add entropy regularization to encourage exploration.

**Entropy Loss Definition**:
$$L^{entropy}(\theta) = -\mathbb{E}[H(\pi_\theta)]$$

Where $H(\pi_\theta) = -\sum_a \pi_\theta(a|s_t) \log \pi_\theta(a|s_t)$ is the entropy.

**Why Negative Entropy?** 
- **High entropy** = more exploration = **good** → we want to **minimize** negative entropy
- **Low entropy** = less exploration = **bad** → minimizing negative entropy **increases** entropy
- This makes entropy loss consistent with our minimization framework

**Total PPO Loss (to minimize)**:
$$L^{TOTAL}(\theta) = L^{CLIP}(\theta) + c_1 L^{V}(\theta) + c_2 L^{entropy}(\theta)$$

Where:
- $L^{CLIP}(\theta)$: Clipped policy loss (already negative, so we minimize it directly)
- $L^{V}(\theta)$: Value function loss
- $L^{entropy}(\theta) = -H(\pi_\theta)$: Negative entropy loss
- $c_1$: Value function loss coefficient (typically 0.5)
- $c_2$: Entropy regularization coefficient (typically 0.01)

**Clean Mathematical Form**: Now all three terms are losses we want to minimize, making the total loss a simple sum without mixed signs.

**Benefits**:
- **Exploration**: Prevents premature convergence to deterministic policies
- **Stability**: Maintains policy diversity throughout training
- **Adaptability**: Automatic annealing as learning progresses
- **Mathematical elegance**: Clean formulation as minimization objective

### 5. 📚 Rollout-Based Updates with Multiple Epochs

**Problem**: Single-step updates waste valuable environment interaction data.

**Solution**: Collect large rollouts from all parallel environments, then train for multiple epochs.

**Data Collection Strategy**:
1. **Collect rollouts**: Gather `rollout_length` steps from all `num_envs` parallel environments
2. **Total transitions**: `rollout_length × num_envs` transitions per update
3. **Multiple epochs**: Train on the same rollout data for K epochs (typically 4-10)
4. **Minibatch updates**: Split rollout into minibatches for efficient GPU utilization
5. **Prevent overfitting**: Clipping and KL penalties prevent over-optimization

### 6. ✂️ Clipped Value Function Loss

**Problem**: Value function updates can also be destructive and unstable.

**Solution**: Clip value function updates similar to policy updates.

**Standard Value Loss**:
$$L^{V}(\theta) = (V_\theta(s_t) - V_t^{target})^2$$

**PPO Clipped Value Loss**:
$$L^{V}(\theta) = \mathbb{E}[\max((V_\theta(s_t) - V_t^{target})^2, (\text{clip}(V_\theta(s_t), V_{old} - \epsilon_v, V_{old} + \epsilon_v) - V_t^{target})^2)]$$

**Why max() and not min()? A Conservative Approach to Clipping**

The maximum operation ensures we **never underestimate the true prediction error** when clipping occurs. Here's the detailed reasoning:

**Case 1: Clipping doesn't constrain the update**
- If $V_{old} - \epsilon_v < V_\theta(s_t) < V_{old} + \epsilon_v$, then clipping has no effect
- The clipped value equals the unclipped value: $\text{clip}(V_\theta(s_t), ...) = V_\theta(s_t)$
- Both loss terms are identical: $\max(\text{same}, \text{same}) = \text{same}$
- Result: Normal, unclipped loss computation

**Case 2: Clipping constrains the update (the critical case)**
- The new value prediction $V_\theta(s_t)$ would move too far from $V_{old}$
- Clipping forces: $V_{clipped} = V_{old} \pm \epsilon_v$ (boundary value)
- Now we have two different loss values to choose from:
  - **Unclipped loss**: $(V_\theta(s_t) - V_t^{target})^2$ (true error)
  - **Clipped loss**: $(V_{clipped} - V_t^{target})^2$ (constrained error)

**The Conservative Principle**: We take the **maximum** (larger) of these two losses because:

1. **Prevent Loss Hiding**: If clipping makes the prediction artificially closer to the target, we don't want to hide this by using the smaller loss
2. **Maintain Learning Signal**: The larger loss preserves the magnitude of the error signal for gradient computation
3. **Avoid Underfitting**: Using min() would encourage the optimizer to prefer clipped updates even when they're less accurate
4. **Consistency Check**: Only allow clipping if it doesn't make the loss artificially small

**Example Scenario**:
- Target return: $V_t^{target} = 100$
- Old value: $V_{old} = 50$
- New prediction: $V_\theta(s_t) = 90$ (moving toward target)
- Clipping bound: $\epsilon_v = 10$
- Clipped value: $V_{clipped} = \min(90, 50 + 10) = 60$

Loss comparison:
- Unclipped loss: $(90 - 100)^2 = 100$
- Clipped loss: $(60 - 100)^2 = 1600$

Using max(): We choose 1600 (the larger loss) because the clipped value is actually **further** from the target. This prevents the clipping from artificially reducing the loss signal.

**Benefits**:
- **Stable value learning**: Prevents large value function updates
- **Conservative clipping**: Only clips when it doesn't hide true error
- **Consistent with policy clipping**: Unified approach to stability
- **Empirical improvement**: Better performance in practice

### 7. 📊 KL Divergence Monitoring (Not Constraining)

**Key Distinction**: Unlike TRPO, PPO doesn't use KL divergence as a **constraint** - it uses it as a **diagnostic tool**.

**What is KL Divergence?**
The Kullback-Leibler divergence measures how much one probability distribution differs from another:

$$KL(\pi_{\theta_{old}}, \pi_\theta) = \mathbb{E}_{s \sim \rho} \mathbb{E}_{a \sim \pi_{\theta_{old}}} \left[ \log \frac{\pi_{\theta_{old}}(a|s)}{\pi_\theta(a|s)} \right]$$

**Intuitive Meaning**:
- **KL = 0**: New policy is identical to old policy
- **KL > 0**: New policy differs from old policy
- **Higher KL**: Larger policy changes

**Why Monitor KL in PPO?**

1. **Training Health Check**: KL divergence tells us how much the policy is changing each update
   - **Healthy range**: 0.001 - 0.01 (modest, stable changes)
   - **Too low**: < 0.0001 (learning stagnation)
   - **Too high**: > 0.1 (potentially destructive updates)

2. **Clipping Effectiveness**: KL helps validate that clipping is working
   - If KL is very high despite clipping, something is wrong
   - If KL is very low, we might be too conservative

3. **Hyperparameter Tuning**: KL guides learning rate and clipping parameter adjustment
   - High KL → reduce learning rate or decrease clip epsilon
   - Low KL → increase learning rate or increase clip epsilon

4. **Early Stopping**: Some implementations use KL divergence for early stopping
   - If KL exceeds a threshold, stop training on current batch
   - Prevents over-optimization on stale data

**PPO's Approach vs TRPO's Approach**:

| Aspect | TRPO | PPO |
|--------|------|-----|
| **KL Usage** | Hard constraint | Diagnostic monitoring |
| **Optimization** | Constrained optimization | Unconstrained with clipping |
| **Computational Cost** | Expensive (second-order) | Cheap (first-order) |
| **Implementation** | Complex conjugate gradient | Simple gradient descent |
| **Robustness** | Sensitive to KL threshold | Robust to hyperparameters |

**Real-World Example**:
In our implementation, you might see:
- **Early training**: KL ≈ 0.01 (policy learning quickly)
- **Mid training**: KL ≈ 0.005 (policy refining)
- **Late training**: KL ≈ 0.001 (policy converging)

**Benefits of KL Monitoring**:
- **Debugging**: Identifies training instabilities early
- **Validation**: Confirms clipping is preventing destructive updates
- **Optimization**: Guides hyperparameter tuning
- **Research**: Enables comparison with TRPO and other methods
- **Zero overhead**: Computed during normal forward pass

**The Bottom Line**: PPO gets TRPO's stability benefits through clipping, but keeps KL monitoring as a "health check" - giving us the best of both worlds with minimal computational overhead.

## 🚀 Why PPO Dominates (2017-2025)

### ✅ PPO's Advantages

1. **Simplicity**: Easy to implement and understand
2. **Stability**: Robust across diverse environments and hyperparameters
3. **Sample Efficiency**: Reuses data effectively through multiple epochs
4. **Computational Efficiency**: First-order optimization, GPU-friendly
5. **Generality**: Works well for both discrete and continuous control
6. **Theoretical Grounding**: Builds on solid policy gradient theory
7. **Empirical Success**: Proven track record in complex domains

### 🏆 PPO's Real-World Impact

**OpenAI's Applications**:
- **ChatGPT**: RLHF (Reinforcement Learning from Human Feedback) training
- **GPT-4**: Large-scale language model alignment
- **OpenAI Five**: Dota 2 championship-level performance
- **Robotics**: Real-world robot control and manipulation

**Industry Adoption**:
- **Default choice**: Most RL practitioners start with PPO
- **Production systems**: Widely used in recommendation systems, game AI, autonomous vehicles
- **Research standard**: Baseline comparison in academic papers

### 📊 PPO vs Alternatives (2025 Perspective)

**PPO vs SAC (Soft Actor-Critic)**:
- **PPO**: Better for discrete actions, more stable, simpler
- **SAC**: Better for continuous control, more sample efficient, more complex

**PPO vs TD3 (Twin Delayed Deep Deterministic)**:
- **PPO**: General-purpose, works with discrete actions
- **TD3**: Continuous control only, more sample efficient in some domains

**PPO vs Modern Methods**:
- **PPO still competitive**: Remains state-of-the-art for many applications
- **Simplicity advantage**: Easier to tune and debug than newer methods
- **Proven reliability**: Extensive empirical validation across domains

## 🔄 PPO Algorithm Overview

**Algorithm: Proximal Policy Optimization (PPO) with Parallel Environments**

---
**Input:** 
- Unified Actor-Critic network with parameters $\theta$
- Number of parallel environments $E$
- Rollout length $T$ (steps per environment)
- Minibatch size $M$
- Number of epochs $K$
- Clipping parameter $\epsilon$
- GAE parameter $\lambda$
- Learning rate $\alpha$
- $c_1$: Value function loss coefficient (typically 0.5)
- $c_2$: Entropy coefficient (typically 0.01)

**Output:** 
- Trained unified network parameters $\theta$

---
**Procedure:**
1. **Initialize** network parameters $\theta$ and $E$ parallel environments
2. **For** iteration $i = 1, 2, ...$ **do:**
3. &nbsp;&nbsp;&nbsp;&nbsp;**For** $t = 1, 2, ..., T$ **do:** *(collect rollout from all environments)*
4. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**For each environment** $e = 1, 2, ..., E$ **do:**
5. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Sample action**: $a_t^{(e)} \sim \pi_\theta(\cdot|s_t^{(e)})$
6. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Execute**: $s_{t+1}^{(e)}, r_{t+1}^{(e)} \leftarrow \text{env}_e.\text{step}(a_t^{(e)})$
7. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Store**: $(s_t^{(e)}, a_t^{(e)}, r_{t+1}^{(e)}, \log \pi_\theta(a_t^{(e)}|s_t^{(e)}), V_\theta(s_t^{(e)}))$
8. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**End For**
9. &nbsp;&nbsp;&nbsp;&nbsp;**End For**
10. &nbsp;&nbsp;&nbsp;&nbsp;**Compute GAE advantages** for all $T \times E$ transitions using parameter $\lambda$
11. &nbsp;&nbsp;&nbsp;&nbsp;**For** epoch $k = 1, 2, ..., K$ **do:**
12. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Shuffle** all $T \times E$ transitions randomly
13. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**For** each minibatch $B$ of size $M$ **do:**
14. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Compute probability ratio**: $r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$
15. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Compute clipped surrogate objective**: $L^{CLIP}(\theta) = -\mathbb{E}_{B}\left[\min(r_t(\theta) A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) A_t)\right]$
16. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Compute clipped value function loss**: $L^{V}(\theta) = \mathbb{E}_{B}[\max((V_\theta(s_t) - V_t^{target})^2, (\text{clip}(V_\theta(s_t), V_{old} - \epsilon_v, V_{old} + \epsilon_v) - V_t^{target})^2)]$
17. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Compute entropy loss**: $L^{entropy}(\theta) = -\mathbb{E}_{B}[H(\pi_\theta)]$
18. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Monitor** KL divergence $KL(\pi_{\theta_{old}}, \pi_\theta)$ (diagnostic only)
19. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Total loss**: $L^{TOTAL}(\theta) = L^{CLIP}(\theta) + c_1 L^{V}(\theta) + c_2 L^{entropy}(\theta)$
20. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Update** $\theta$ using gradient descent: $\theta \leftarrow \theta - \alpha \nabla_\theta L^{TOTAL}(\theta)$
21. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**End For**
22. &nbsp;&nbsp;&nbsp;&nbsp;**End For**
23. **End For**

---

### 📐 PPO Loss Functions Detailed

**Clipped Surrogate Objective Loss**:
$$L^{CLIP}(\theta) = -\mathbb{E}\left[\sum_{t=0}^{T-1} \min(r_t(\theta) A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) A_t)\right]$$

Where:
- $r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$ is the probability ratio
- $A_t$ is the advantage estimate
- $\text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon)$ constrains the ratio to $[1-\epsilon, 1+\epsilon]$
- The $\min()$ operation implements the conservative update principle

**Clipped Value Function Loss**:
$$L^{V}(\theta) = \mathbb{E}[\max((V_\theta(s_t) - V_t^{target})^2, (\text{clip}(V_\theta(s_t), V_{old} - \epsilon_v, V_{old} + \epsilon_v) - V_t^{target})^2)]$$

Where:
- $V_\theta(s_t)$ is the current value prediction
- $V_t^{target}$ is the target value (GAE-computed return)
- $V_{old}$ is the value prediction from the old network
- $\epsilon_v$ is the value clipping parameter
- The $\max()$ operation ensures conservative clipping (prevents loss hiding)

**Entropy Loss**:
$$L^{entropy}(\theta) = -\mathbb{E}[H(\pi_\theta)] = -\mathbb{E}\left[-\sum_a \pi_\theta(a|s_t) \log \pi_\theta(a|s_t)\right]$$

Where $H(\pi_\theta)$ is the entropy of the policy distribution, encouraging exploration.

In [ ]:
import numpy as np
import torch
import torch.optim as optim
from collections import deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

# Import our RL utilities including the ActorCriticNetwork
from rl_utils import (
    set_seeds,
    ActorCriticNetwork,
    create_env_with_wrappers,
    plot_training_results,
    plot_variance_analysis,
)
from rl_utils.visualization import (
    get_moving_average,
    plot_ppo_training_results,
    plot_ppo_variance_analysis,
)

# Create configuration
CONFIG = {
    "seed": 42,
    "episodes": 1000,
    "gamma": 0.99,
    "lr": 5e-4,
    "device": "cuda",
    "window_length": 50,
    "target_score": 200,  # LunarLander-v3 target score
    # Environment: LunarLander-v3 only
    "env_id": "LunarLander-v3",
    "env_kwargs": {
        "gravity": -10.0,
        "enable_wind": False,
        "wind_power": 15.0,
        "turbulence_power": 1.5,
    },
    # Video Recording Config
    "record_videos": True,
    "video_folder": "videos",
    "num_videos": 9,  # Number of videos to record during training
    "record_test_videos": True,
    # Neural Network Config
    "network": {
        "fc_out_features": [64, 64],  # Shared features
        "actor_features": [32],  # Actor-specific layers after shared
        "critic_features": [32],  # Critic-specific layers after shared
        "activation": "SiLU",
        "use_layer_norm": True,
        "dropout_rate": 0.0,  # No dropout for stability
    },
    # PPO-Specific Parameters
    "rollout_length": 2048,  # Steps per rollout
    "minibatch_size": 64,    # Minibatch size for updates
    "epochs": 10,            # Training epochs per rollout
    "clip_epsilon": 0.2,     # Clipping parameter
    "value_clip_epsilon": 0.2,  # Value function clipping
    "critic_loss_coeff": 0.5,   # Weight for critic loss
    "entropy_coeff": 0.01,      # Weight for entropy bonus
    "gae_lambda": 0.95,         # GAE parameter
    "max_grad_norm": 0.5,       # Maximum gradient norm for clipping
}

set_seeds(CONFIG["seed"])
print(f"🎲 Global random seeds set to {CONFIG['seed']} for reproducible results")
print(
    f"📝 Environment episodes will use seeds {CONFIG['seed']} + episode_number for varied but reproducible episodes"
)

In [ ]:
class PPOAgent:
    """Proximal Policy Optimization agent with all key innovations."""

    def __init__(self, network, config):
        """
        Initialize PPO agent.

        Args:
            network: ActorCriticNetwork instance
            config: Configuration dictionary
        """
        self.network = network.to(config["device"])
        self.device = config["device"]
        self.gamma = config["gamma"]
        self.rollout_length = config["rollout_length"]
        self.minibatch_size = config["minibatch_size"]
        self.epochs = config["epochs"]
        self.clip_epsilon = config["clip_epsilon"]
        self.value_clip_epsilon = config["value_clip_epsilon"]
        self.critic_loss_coeff = config["critic_loss_coeff"]
        self.entropy_coeff = config["entropy_coeff"]
        self.gae_lambda = config["gae_lambda"]
        self.max_grad_norm = config["max_grad_norm"]
        self.window_size = config.get("window_length")

        # Single optimizer for all network parameters
        self.optimizer = optim.Adam(
            self.network.parameters(),
            lr=config["lr"],
        )

        # Print detailed network information
        print(f"📊 PPO NETWORK DETAILS:")
        self.network.print_network_info()
        print(f"🎯 Rollout Length: {self.rollout_length}")
        print(f"📦 Minibatch Size: {self.minibatch_size}")
        print(f"🔁 Training Epochs: {self.epochs}")
        print(f"✂️ Clip Epsilon: {self.clip_epsilon}")
        print(f"🌟 Entropy Coefficient: {self.entropy_coeff}")
        print(f"🎲 GAE Lambda: {self.gae_lambda}")
        print(f"🎓 Learning Rate: {config['lr']}")
        print(f"✂️ Max Gradient Norm: {self.max_grad_norm}")
        print(f"⚖️ Critic Loss Coefficient: {self.critic_loss_coeff}")

        # Rollout buffer storage
        self.reset_rollout_buffer()

        # Variance and performance tracking
        self.gradient_norms = []
        self.episode_scores = []  # Raw undiscounted episode scores (only thing we can track in PPO)
        self.score_variance_history = []

        # Update step tracking
        self.update_step = 0
        self.update_steps_history = []

        # Loss component tracking
        self.loss_history = {
            "actor_loss": [],
            "critic_loss": [],
            "entropy_loss": [],
            "total_loss": [],
        }

        # PPO-specific tracking
        self.rollout_count = 0
        self.steps_collected = 0
        self.policy_updates = 0
        self.clip_fraction_history = []
        self.kl_divergence_history = []

    def reset_rollout_buffer(self):
        """Reset rollout buffer for new collection."""
        self.states = []
        self.actions = []
        self.rewards = []
        self.log_probs = []
        self.values = []
        self.dones = []
        self.advantages = []
        self.returns = []
        self.current_episode_return = 0.0
        self.episode_step_count = 0

    def select_action(self, state):
        """Select an action and store necessary data."""
        state = torch.as_tensor(state, dtype=torch.float32, device=self.device)

        with torch.no_grad():
            # Get policy distribution and value estimate
            dist, value = self.network(state)
            action = dist.sample()

            # Store log probability and value prediction
            if self.network.is_continuous:
                log_prob = dist.log_prob(action).sum(-1)
                action_to_env = self.network.clip_action(action).flatten()
            else:
                log_prob = dist.log_prob(action)
                action_to_env = action.item()

        # Store in rollout buffer
        self.states.append(state)
        self.actions.append(action)
        self.log_probs.append(log_prob)
        self.values.append(value)

        self.steps_collected += 1
        self.episode_step_count += 1

        return action_to_env

    def store_transition(self, reward, done):
        """Store reward and done flag."""
        self.rewards.append(reward)
        self.dones.append(done)
        self.current_episode_return += reward

        # If episode ended, store episode score
        if done:
            self.episode_scores.append(self.current_episode_return)
            self.current_episode_return = 0.0
            self.episode_step_count = 0

    def is_rollout_complete(self):
        """Check if rollout is complete."""
        return len(self.states) >= self.rollout_length

    def compute_gae_advantages(self, next_value=0.0):
        """Compute Generalized Advantage Estimation."""
        # Convert to tensors
        rewards = torch.tensor(self.rewards, dtype=torch.float32, device=self.device)
        values = torch.stack(self.values)
        dones = torch.tensor(self.dones, dtype=torch.float32, device=self.device)  # Convert to float

        # Compute advantages using GAE
        advantages = []
        gae = 0.0
        
        # Work backwards through the episode
        for t in reversed(range(len(rewards))):
            if t == len(rewards) - 1:
                # Last step - use next_value from environment or 0 if terminal
                next_value_t = next_value if not self.dones[t] else 0.0
            else:
                next_value_t = values[t + 1]
            
            # TD error
            delta = rewards[t] + self.gamma * next_value_t * (1 - dones[t]) - values[t]
            
            # GAE calculation
            gae = delta + self.gamma * self.gae_lambda * (1 - dones[t]) * gae
            advantages.insert(0, gae)

        advantages = torch.tensor(advantages, dtype=torch.float32, device=self.device)
        returns = advantages + values.detach()

        return advantages, returns

    def update_policy(self, next_state=None):
        """Update policy using PPO algorithm."""
        if len(self.states) < self.minibatch_size:
            return {"actor_loss": 0.0, "critic_loss": 0.0, "entropy_loss": 0.0, "total_loss": 0.0}, 0.0

        self.rollout_count += 1

        # Get next state value for GAE calculation
        if next_state is not None:
            next_state_tensor = torch.as_tensor(next_state, dtype=torch.float32, device=self.device)
            with torch.no_grad():
                _, next_value = self.network(next_state_tensor)
                next_value = next_value.item()
        else:
            next_value = 0.0

        # Compute advantages and returns using GAE
        advantages, returns = self.compute_gae_advantages(next_value)

        # Track score variance
        if len(self.episode_scores) >= self.window_size:
            recent_scores = self.episode_scores[-self.window_size:]
            score_variance = np.var(recent_scores)
            self.score_variance_history.append(score_variance)

        # Convert rollout data to tensors
        states = torch.stack(self.states)
        actions = torch.stack(self.actions)
        old_log_probs = torch.stack(self.log_probs)
        old_values = torch.stack(self.values)

        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # Store for reuse across epochs
        rollout_data = {
            'states': states,
            'actions': actions,
            'old_log_probs': old_log_probs,
            'old_values': old_values,
            'advantages': advantages,
            'returns': returns,
        }

        # Train for multiple epochs
        total_actor_loss = 0.0
        total_critic_loss = 0.0
        total_entropy_loss = 0.0
        total_loss = 0.0
        total_grad_norm = 0.0
        clip_fraction = 0.0
        kl_divergence = 0.0
        num_updates = 0

        for epoch in range(self.epochs):
            # Shuffle data for each epoch
            indices = torch.randperm(len(states))
            
            for start_idx in range(0, len(states), self.minibatch_size):
                end_idx = min(start_idx + self.minibatch_size, len(states))
                batch_indices = indices[start_idx:end_idx]
                
                if len(batch_indices) < self.minibatch_size // 2:  # Skip very small batches
                    continue

                # Extract minibatch
                batch_states = states[batch_indices]
                batch_actions = actions[batch_indices]
                batch_old_log_probs = old_log_probs[batch_indices]
                batch_old_values = old_values[batch_indices]
                batch_advantages = advantages[batch_indices]
                batch_returns = returns[batch_indices]

                # Forward pass
                dist, values = self.network(batch_states)
                
                # Compute new log probabilities
                if self.network.is_continuous:
                    new_log_probs = dist.log_prob(batch_actions).sum(-1)
                else:
                    new_log_probs = dist.log_prob(batch_actions)

                # Compute probability ratio
                ratio = torch.exp(new_log_probs - batch_old_log_probs)

                # Clipped surrogate objective (already negative, so this is L^CLIP to minimize)
                surr1 = ratio * batch_advantages
                surr2 = torch.clamp(ratio, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * batch_advantages
                actor_loss = -torch.min(surr1, surr2).mean()

                # Clipped value function loss
                value_pred_clipped = batch_old_values + torch.clamp(
                    values - batch_old_values, -self.value_clip_epsilon, self.value_clip_epsilon
                )
                value_loss_1 = (values - batch_returns).pow(2)
                value_loss_2 = (value_pred_clipped - batch_returns).pow(2)
                critic_loss = 0.5 * torch.max(value_loss_1, value_loss_2).mean()

                # critic loss calculation without clipping. try this.
                # critic_loss = 0.5 * (values - batch_returns).pow(2).mean()  

                # Entropy loss: L^entropy = -H(π) (negative entropy to minimize)
                entropy = dist.entropy()
                if entropy.dim() > 1:
                    entropy = entropy.sum(-1)
                entropy_loss = -entropy.mean()  # This is L^entropy (negative entropy)

                # Total loss: L^TOTAL = L^CLIP + c1*L^VF + c2*L^entropy
                total_loss_batch = actor_loss + self.critic_loss_coeff * critic_loss + self.entropy_coeff * entropy_loss

                # Update network
                self.optimizer.zero_grad()
                total_loss_batch.backward()

                # Record gradient norm BEFORE clipping
                grad_norm = 0.0
                for param in self.network.parameters():
                    if param.grad is not None:
                        param_norm = param.grad.data.norm(2)
                        grad_norm += param_norm.item() ** 2
                grad_norm = grad_norm ** 0.5

                # Apply gradient clipping
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), self.max_grad_norm)

                self.optimizer.step()

                # Accumulate statistics
                total_actor_loss += actor_loss.item()
                total_critic_loss += critic_loss.item()
                total_entropy_loss += entropy_loss.item()
                total_loss += total_loss_batch.item()
                total_grad_norm += grad_norm
                
                # Track clipping and KL divergence
                with torch.no_grad():
                    clipped = torch.abs(ratio - 1.0) > self.clip_epsilon
                    clip_fraction += clipped.float().mean().item()
                    kl_divergence += (batch_old_log_probs - new_log_probs).mean().item()

                num_updates += 1

        # Average statistics
        if num_updates > 0:
            avg_actor_loss = total_actor_loss / num_updates
            avg_critic_loss = total_critic_loss / num_updates
            avg_entropy_loss = total_entropy_loss / num_updates
            avg_total_loss = total_loss / num_updates
            avg_grad_norm = total_grad_norm / num_updates
            avg_clip_fraction = clip_fraction / num_updates
            avg_kl_divergence = kl_divergence / num_updates
        else:
            avg_actor_loss = avg_critic_loss = avg_entropy_loss = avg_total_loss = avg_grad_norm = 0.0
            avg_clip_fraction = avg_kl_divergence = 0.0

        # Store statistics
        self.update_step += 1
        self.policy_updates += num_updates
        self.gradient_norms.append(avg_grad_norm)
        self.loss_history["actor_loss"].append(avg_actor_loss)
        self.loss_history["critic_loss"].append(avg_critic_loss)
        self.loss_history["entropy_loss"].append(avg_entropy_loss)
        self.loss_history["total_loss"].append(avg_total_loss)
        self.update_steps_history.append(self.update_step)
        self.clip_fraction_history.append(avg_clip_fraction)
        self.kl_divergence_history.append(avg_kl_divergence)

        # Reset rollout buffer
        self.reset_rollout_buffer()

        return {
            "actor_loss": avg_actor_loss,
            "critic_loss": avg_critic_loss,
            "entropy_loss": avg_entropy_loss,
            "total_loss": avg_total_loss,
        }, avg_grad_norm

    def get_variance_stats(self):
        """Get variance statistics for analysis."""
        if len(self.episode_scores) < 2:
            return {
                "gradient_norm_mean": 0.0,
                "gradient_norm_std": 0.0,
                "score_mean": 0.0,
                "score_std": 0.0,
                "recent_score_variance": 0.0,
                "clip_fraction_mean": 0.0,
                "kl_divergence_mean": 0.0,
            }

        return {
            "gradient_norm_mean": np.mean(self.gradient_norms),
            "gradient_norm_std": np.std(self.gradient_norms),
            "score_mean": np.mean(self.episode_scores),
            "score_std": np.std(self.episode_scores),
            "recent_score_variance": (
                self.score_variance_history[-1]
                if self.score_variance_history
                else 0.0
            ),
            "clip_fraction_mean": np.mean(self.clip_fraction_history) if self.clip_fraction_history else 0.0,
            "kl_divergence_mean": np.mean(self.kl_divergence_history) if self.kl_divergence_history else 0.0,
        }

    def get_ppo_stats(self):
        """Get PPO-specific statistics."""
        return {
            "rollout_count": self.rollout_count,
            "steps_collected": self.steps_collected,
            "policy_updates": self.policy_updates,
            "updates_per_rollout": self.policy_updates / max(1, self.rollout_count),
            "steps_per_update": self.steps_collected / max(1, self.policy_updates),
        }

In [ ]:
def train_ppo(is_continuous, config):
    """Main training loop for the PPO agent."""
    action_type = "Continuous" if is_continuous else "Discrete"
    print(f"\n{'='*70}")
    print(f"PROXIMAL POLICY OPTIMIZATION (PPO) - {action_type.upper()}")
    print(f"{'='*70}")
    
    # Calculate video recording interval
    video_record_interval = max(1, config["episodes"] // config["num_videos"])
    print(f"📹 Recording {config['num_videos']} videos every {video_record_interval} episodes")
    
    # Create algorithm-specific video folder
    video_folder = f"videos/PPO_{action_type.lower()}"
    config_with_videos = config.copy()
    config_with_videos["video_folder"] = video_folder
    config_with_videos["video_record_interval"] = video_record_interval
    
    # Create Environment
    env = create_env_with_wrappers(
        config_with_videos, 
        is_continuous, 
        record_videos=True, 
        video_prefix=f"ppo_{action_type.lower()}",
        cleanup_existing=True
    )
    
    # Get observation dimension and space
    dummy_obs, _ = env.reset()
    observation_dim = len(dummy_obs)
    
    # Create Actor-Critic Network and Agent
    print(f"\n🏗️ CREATING {action_type.upper()} ACTOR-CRITIC NETWORK:")
    network = ActorCriticNetwork(
        observation_dim=observation_dim,
        action_space=env.action_space,
        is_continuous=is_continuous,
        network_config=config["network"]
    )
    
    print(f"\n🤖 INITIALIZING {action_type.upper()} PPO AGENT:")
    agent = PPOAgent(network, config)
    
    # Training Loop
    scores = []
    scores_window = deque(maxlen=config["window_length"])
    start_time = time.time()
    
    print(f"\n🚀 STARTING {action_type.upper()} PPO TRAINING...")
    
    # Initialize environment
    state, _ = env.reset(seed=config["seed"])
    episode_count = 0
    
    # Use tqdm for progress tracking
    pbar = tqdm(total=config["episodes"], desc="Training", unit="episode")
    
    while episode_count < config["episodes"]:
        # Collect rollout
        while not agent.is_rollout_complete():
            action = agent.select_action(state)
            
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            
            agent.store_transition(reward, done)
            
            state = next_state
            
            if done:
                # Episode finished
                episode_count += 1
                episode_reward = agent.episode_scores[-1] if agent.episode_scores else 0.0
                scores.append(episode_reward)
                scores_window.append(episode_reward)
                
                # Reset environment
                state, _ = env.reset(seed=config["seed"] + episode_count)
                
                # Update progress bar
                avg_score_window = np.mean(scores_window) if len(scores_window) > 0 else 0.0
                
                # Get latest loss values and metrics
                actor_loss = agent.loss_history["actor_loss"][-1] if agent.loss_history["actor_loss"] else 0.0
                critic_loss = agent.loss_history["critic_loss"][-1] if agent.loss_history["critic_loss"] else 0.0
                entropy_loss = agent.loss_history["entropy_loss"][-1] if agent.loss_history["entropy_loss"] else 0.0
                total_loss = agent.loss_history["total_loss"][-1] if agent.loss_history["total_loss"] else 0.0
                grad_norm = agent.gradient_norms[-1] if agent.gradient_norms else 0.0
                
                # Get PPO-specific metrics
                clip_fraction = agent.clip_fraction_history[-1] if agent.clip_fraction_history else 0.0
                kl_divergence = agent.kl_divergence_history[-1] if agent.kl_divergence_history else 0.0
                
                pbar.set_description(
                    f"Ep {episode_count:4d} | "
                    f"Score: {episode_reward:6.1f} | "
                    f"AvgScore({config['window_length']}): {avg_score_window:6.1f} | "
                    f"Rollouts: {agent.rollout_count:3d} | "
                    f"Updates: {agent.policy_updates:4d} | "
                    f"ActorLoss: {actor_loss:7.4f} | "
                    f"CriticLoss: {critic_loss:7.4f} | "
                    f"EntLoss: {entropy_loss:7.4f} | "
                    f"TotalLoss: {total_loss:7.4f} | "
                    f"GradNorm: {grad_norm:6.4f} | "
                    f"ClipFrac: {clip_fraction:5.3f} | "
                    f"KL: {kl_divergence:7.4f}"
                )
                pbar.update(1)
                
                # Handle video display
                if episode_count % video_record_interval == 0 and config["record_videos"]:
                    from rl_utils.environment import display_latest_video
                    pbar.write(f"\nVideo recorded at episode {episode_count}")
                    display_latest_video(
                        config_with_videos["video_folder"], 
                        f"ppo_{action_type.lower()}", 
                        episode_count
                    )
                
                # Stop if we've reached the target number of episodes
                if episode_count >= config["episodes"]:
                    break
        
        # Update policy when rollout is complete
        if agent.is_rollout_complete():
            loss_dict, grad_norm = agent.update_policy(state)
            
            # Update progress information with latest metrics
            avg_score_window = np.mean(scores_window) if len(scores_window) > 0 else 0.0
            actor_loss = loss_dict.get('actor_loss', 0.0)
            critic_loss = loss_dict.get('critic_loss', 0.0)
            entropy_loss = loss_dict.get('entropy_loss', 0.0)
            total_loss = loss_dict.get('total_loss', 0.0)
            
            # Get PPO-specific stats
            variance_stats = agent.get_variance_stats()
            clip_fraction = variance_stats.get('clip_fraction_mean', 0.0)
            kl_divergence = variance_stats.get('kl_divergence_mean', 0.0)
            
            pbar.set_description(
                f"Ep {episode_count:4d} | "
                f"Score: {scores[-1] if scores else 0.0:6.1f} | "
                f"AvgScore({config['window_length']}): {avg_score_window:6.1f} | "
                f"Rollouts: {agent.rollout_count:3d} | "
                f"Updates: {agent.policy_updates:4d} | "
                f"ActorLoss: {actor_loss:7.4f} | "
                f"CriticLoss: {critic_loss:7.4f} | "
                f"EntLoss: {entropy_loss:7.4f} | "
                f"TotalLoss: {total_loss:7.4f} | "
                f"GradNorm: {grad_norm:6.4f} | "
                f"ClipFrac: {clip_fraction:5.3f} | "
                f"KL: {kl_divergence:7.4f}"
            )
    
    pbar.close()
    env.close()
    
    elapsed_time = time.time() - start_time
    final_window_size = min(config["window_length"], len(scores))
    final_performance = np.mean(scores[-final_window_size:]) if final_window_size > 0 else 0.0
    
    # Print PPO-specific statistics
    ppo_stats = agent.get_ppo_stats()
    print(f"\n{action_type} PPO training completed in {elapsed_time:.1f} seconds!")
    print(f"Final performance: {final_performance:.2f} (last {final_window_size} episodes)")
    print(f"Total rollouts: {ppo_stats['rollout_count']}")
    print(f"Total policy updates: {ppo_stats['policy_updates']}")
    print(f"Steps collected: {ppo_stats['steps_collected']}")
    print(f"Updates per rollout: {ppo_stats['updates_per_rollout']:.1f}")
    print(f"Steps per update: {ppo_stats['steps_per_update']:.1f}")
    
    return scores, agent.loss_history, agent

In [ ]:
# --- DISCRETE ACTION SPACE: PPO ---
print("Starting PPO training with DISCRETE actions...")

discrete_ppo_scores, discrete_ppo_losses, discrete_ppo_agent = train_ppo(
    is_continuous=False, 
    config=CONFIG
)

In [ ]:
# Plot results for discrete PPO
plot_ppo_training_results(
    discrete_ppo_scores, 
    discrete_ppo_agent.loss_history, 
    CONFIG, 
    "Discrete", 
    algorithm_name="PPO"
)

# Show variance analysis
plot_ppo_variance_analysis(
    discrete_ppo_agent, 
    discrete_ppo_scores, 
    "Discrete", 
    CONFIG, 
    algorithm_name="PPO"
)

# PPO-specific analysis
variance_stats = discrete_ppo_agent.get_variance_stats()
ppo_stats = discrete_ppo_agent.get_ppo_stats()

print(f"\n🔍 DISCRETE PPO ANALYSIS:")
print(f"Clip fraction: {variance_stats['clip_fraction_mean']:.3f} (target: ~{CONFIG['clip_epsilon']:.1f})")
print(f"KL divergence: {variance_stats['kl_divergence_mean']:.4f}")
print(f"Data efficiency: {ppo_stats['steps_per_update']:.1f} env steps per policy update (lower the better)")

# Training completion message
final_window_size = min(CONFIG["window_length"], len(discrete_ppo_scores))
final_avg = np.mean(discrete_ppo_scores[-final_window_size:]) if final_window_size > 0 else 0.0
print(f"\n✅ DISCRETE PPO TRAINING COMPLETED!")
print(f"Final average score (last {final_window_size} episodes): {final_avg:.2f}")
print(f"Network parameters: {discrete_ppo_agent.network.get_param_count():,}")

In [ ]:
# --- CONTINUOUS ACTION SPACE: PPO ---
print("Starting PPO training with CONTINUOUS actions...")

continuous_ppo_scores, continuous_ppo_losses, continuous_ppo_agent = train_ppo(
    is_continuous=True, 
    config=CONFIG
)

In [ ]:
# Plot results for continuous PPO
plot_ppo_training_results(
    continuous_ppo_scores, 
    continuous_ppo_agent.loss_history, 
    CONFIG, 
    "Continuous", 
    algorithm_name="PPO"
)

# Show variance analysis
plot_ppo_variance_analysis(
    continuous_ppo_agent, 
    continuous_ppo_scores, 
    "Continuous", 
    CONFIG, 
    algorithm_name="PPO"
)

# PPO-specific analysis
variance_stats = continuous_ppo_agent.get_variance_stats()
ppo_stats = continuous_ppo_agent.get_ppo_stats()

print(f"\n🔍 CONTINUOUS PPO ANALYSIS:")
print(f"Clip fraction: {variance_stats['clip_fraction_mean']:.3f} (target: ~{CONFIG['clip_epsilon']:.1f})")
print(f"KL divergence: {variance_stats['kl_divergence_mean']:.4f}")
print(f"Data efficiency: {ppo_stats['steps_per_update']:.1f} env steps per policy update (lower the better)")

# Training completion message
final_window_size = min(CONFIG["window_length"], len(continuous_ppo_scores))
final_avg = np.mean(continuous_ppo_scores[-final_window_size:]) if final_window_size > 0 else 0.0
print(f"\n✅ CONTINUOUS PPO TRAINING COMPLETED!")
print(f"Final average score (last {final_window_size} episodes): {final_avg:.2f}")
print(f"Network parameters: {continuous_ppo_agent.network.get_param_count():,}")

In [ ]:
# --- COMPREHENSIVE ANALYSIS: PPO Performance and Efficiency ---
import matplotlib.pyplot as plt
from rl_utils.visualization import get_moving_average

print(f"\n{'='*80}")
print(f"COMPREHENSIVE ANALYSIS: PPO Performance and Efficiency")
print(f"{'='*80}")

# Collect experiment results
experiments = [
    ("Discrete PPO", discrete_ppo_scores, discrete_ppo_agent),
    ("Continuous PPO", continuous_ppo_scores, continuous_ppo_agent),
]

print(f"\n📊 FINAL PERFORMANCE COMPARISON (last {CONFIG['window_length']} episodes):")
print(f"{'Method':<20} {'Final Score':<12} {'Score Std':<10} {'Rollouts':<10} {'Updates':<8} {'Up/Ep':<8} {'ClipFrac':<10} {'Parameters':<12}")
print("-" * 105)

for name, scores, agent in experiments:
    final_window_size = min(CONFIG["window_length"], len(scores))
    final_score = np.mean(scores[-final_window_size:]) if final_window_size > 0 else 0.0
    
    stats = agent.get_variance_stats()
    ppo_stats = agent.get_ppo_stats()
    param_count = agent.network.get_param_count()
    
    # Calculate updates per episode
    total_updates = ppo_stats['policy_updates']
    updates_per_episode = total_updates / len(scores) if len(scores) > 0 else 0.0
    
    print(f"{name:<20} {final_score:<12.1f} {stats['score_std']:<10.1f} {ppo_stats['rollout_count']:<10} {total_updates:<8} {updates_per_episode:<8.1f} {stats['clip_fraction_mean']:<10.3f} {param_count:<12,}")

print(f"\n📈 PPO-SPECIFIC ANALYSIS:")
for name, scores, agent in experiments:
    stats = agent.get_variance_stats()
    ppo_stats = agent.get_ppo_stats()
    recent_score_var = stats.get('recent_score_variance', 0.0)
    
    print(f"{name}:")
    print(f"  Score variance (last {CONFIG['window_length']} episodes): {recent_score_var:.1f}")
    print(f"  Clip fraction: {stats['clip_fraction_mean']:.3f} (healthy: 0.1-0.3)")
    print(f"  KL divergence: {stats['kl_divergence_mean']:.4f} (healthy: <0.01)")
    print(f"  Data efficiency: {ppo_stats['steps_per_update']:.1f} env steps per policy update (lower the better)")

# Create PPO-specific analysis plots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))
fig.suptitle('PPO Analysis: Performance and Training Dynamics', fontsize=16)

colors = ['blue', 'red']
smoothing_window = CONFIG["window_length"]

# 1. Performance comparison (episodes)
for i, (name, scores, agent) in enumerate(experiments):
    if len(scores) >= smoothing_window:
        smoothed, offset = get_moving_average(scores, window=smoothing_window)
        episodes = range(offset + 1, offset + 1 + len(smoothed))
        ax1.plot(episodes, smoothed, label=name, color=colors[i], linewidth=2)

ax1.axhline(y=CONFIG["target_score"], color='g', linestyle='--', label=f'Target ({CONFIG["target_score"]})', alpha=0.7)
ax1.set_xlabel('Episode')
ax1.set_ylabel(f'Score ({smoothing_window}-episode avg)')
ax1.set_title('PPO Performance Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Clip fraction over time (rollouts)
for i, (name, scores, agent) in enumerate(experiments):
    if len(agent.clip_fraction_history) >= smoothing_window:
        smoothed, offset = get_moving_average(agent.clip_fraction_history, window=smoothing_window)
        rollouts = range(offset + 1, offset + 1 + len(smoothed))
        ax2.plot(rollouts, smoothed, label=name, color=colors[i], linewidth=2)

ax2.axhline(y=CONFIG["clip_epsilon"], color='orange', linestyle='--', label=f'Clip Epsilon ({CONFIG["clip_epsilon"]})', alpha=0.7)
ax2.set_xlabel('Rollout')
ax2.set_ylabel('Clip Fraction')
ax2.set_title('Policy Update Clipping Over Time')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. KL divergence over time (rollouts)
for i, (name, scores, agent) in enumerate(experiments):
    if len(agent.kl_divergence_history) >= smoothing_window:
        smoothed, offset = get_moving_average(agent.kl_divergence_history, window=smoothing_window)
        rollouts = range(offset + 1, offset + 1 + len(smoothed))
        ax3.plot(rollouts, smoothed, label=name, color=colors[i], linewidth=2)

ax3.set_xlabel('Rollout')
ax3.set_ylabel('KL Divergence')
ax3.set_title('Policy Change Over Time')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Loss components over time (rollouts)
for i, (name, scores, agent) in enumerate(experiments):
    if len(agent.loss_history["total_loss"]) >= smoothing_window:
        # Plot total loss
        smoothed, offset = get_moving_average(agent.loss_history["total_loss"], window=smoothing_window)
        rollouts = range(offset + 1, offset + 1 + len(smoothed))
        ax4.plot(rollouts, smoothed, label=f'{name} Total', color=colors[i], linewidth=2)
        
        # Plot actor loss with transparency
        if len(agent.loss_history["actor_loss"]) >= smoothing_window:
            actor_smoothed, _ = get_moving_average(agent.loss_history["actor_loss"], window=smoothing_window)
            ax4.plot(rollouts, actor_smoothed, label=f'{name} Actor', color=colors[i], alpha=0.6, linestyle='--')
        
        # Plot critic loss with transparency
        if len(agent.loss_history["critic_loss"]) >= smoothing_window:
            critic_smoothed, _ = get_moving_average(agent.loss_history["critic_loss"], window=smoothing_window)
            ax4.plot(rollouts, critic_smoothed, label=f'{name} Critic', color=colors[i], alpha=0.6, linestyle=':')

ax4.set_xlabel('Rollout')
ax4.set_ylabel('Loss Value')
ax4.set_title('Loss Components Over Time')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()